## Notebook for collecting metadata, processing imaging data through suite2p and analysing trial responses

In [ ]:
%run ./rob_setup_notebook.ipynb

import utils.interareal_analysis as ia

### Define sessions to run s2p or analysis on

In [ ]:
ss_id = '1PgSXs6BVHe9dACWEydykucN74rirNfOY4Vr_AmLcCdY'
sheet_names = [
#     '2019-06-11_J048',
#     '2019-06-13_J052',
#     '2019-06-25_RL048',
#     '2019-07-15_RL052',
#     '2019-08-14_J058',
#     '2019-08-14_J059',
#     '2019-08-15_RL055',
#     '2019-11-14_J061',
#     '2019-11-14_J063',
#     '2019-11-15_J064',
#     '2019-11-18_J065',
#     '2020-02-27_RL075',
#     '2020-03-03_RL074',
#     '2020-03-05_RL070',
#     '2020-03-05_RL072',
#     '2020-03-12_RL070',
#     '2020-03-16_RL074',
#     '2020-03-19_RL075',
#     '2020-06-03_RL086',
#     '2020-06-08_RL086',
#     '2020-06-15_RL086',
#     '2020-09-03_RL091',
#     '2020-09-04_RL093',
#     '2020-09-08_RL099',
#     '2020-09-09_RL100',
#     '2020-09-14_RL101',
#     '2020-09-15_RL102',
#     '2020-09-16_RL103',
#     '2020-10-02_RL108',
#     '2020-10-12_RL109',
#     '2020-10-29_RL111',
#     '2020-12-10_RL115',
#     '2021-01-06_RL120',
#     '2021-01-20_RL121',
#     '2021-01-20_RL125', 
#     '2021-01-20_RL124', 
#     '2021-01-28_RL124', 
#     '2021-02-18_RL127', 
#     '2021-02-22_RL128', 
#     '2021-02-23_RL129', 
#     '2021-02-24_RL132', 
#     '2021-02-25_RL131', 
#     '2021-03-02_RL130'
              ]

### Run s2p on defined sessions (commented-out to not overwrite data by accident)

In [ ]:
# for sheet_name in sheet_names:
    
#     # Initialise the class
#     ses_obj = ia.interarealProcessing(ss_id, sheet_name, qnap_data_path)
    
#     # Run s2p on this experiment (if not already done)
#     ses_obj.s2pRun(tau=0.7) # tau = 0.7 for fast, 1.25 for slow
    
#     # Pickle the object output to save it for analysis
#     pkl_path = os.path.join(pkl_folder, sheet_name + '.pkl')
    
#     with open(pkl_path, 'wb') as f:
#         pickle.dump(ses_obj, f)

### Run sessions through analysis pipeline (after s2p ROI curation)

In [ ]:
trial_sig_calc = 'dff' # how to calculate trial significance ('dff' or 'dfsf' = change in f over std deviation of f)

for sheet_name in sheet_names:
    
    # Initialise the class
    ses_obj = ia.interarealProcessing(ss_id, sheet_name, qnap_data_path)
    
    # Get the range of frames that are attributed to each experiment (from the concatenated t-series)
    ses_obj.getFrameRanges()
    
    # Process the s2p data (after manual curation)
    ses_obj.photostim_r.s2pAnalysis(s2_borders_path, trial_sig_calc)
    ses_obj.photostim_s.s2pAnalysis(s2_borders_path, trial_sig_calc)
    ses_obj.whisker_stim.s2pAnalysis(s2_borders_path, trial_sig_calc)
    
    # Add sham photostim metadata
    if ses_obj.spont.n_frames > 0:
        ses_obj.addShamPhotostim()
        
    ses_obj.spont.s2pAnalysis(s2_borders_path, trial_sig_calc)
    
    ses_obj.whiskerTargets()
    
    # Pickle the object output to save it for analysis
    pkl_path = os.path.join(pkl_folder, sheet_name + '.pkl')
    
    with open(pkl_path, 'wb') as f:
        pickle.dump(ses_obj, f)        